# Homework 2

## FINM 37500: Fixed Income Derivatives

### Mark Hendricks

#### Winter 2025

***

# 1.

## Swaption Vol Data

The file `data/swaption_vol_data_2025-01-31.xlsx` has market data on the implied volatility skews for swaptions. Note that it has several columns:
* `expry`: expiration of the swaption
* `tenor`: tenor of the underlying swap
* `model`: the model by which the volatility is quoted. (All are Black.)
* `-200`, `-100`, etc.: The strike listed as difference from ATM strike (bps). Note that ATM is considered to be the **forward swapa rate** which you can calculate.

In [31]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
from scipy.optimize import brentq

SWAPTION_FILE_PATH = "../data/swaption_vol_data_2025-01-31.xlsx"

swap_df = pd.read_excel(SWAPTION_FILE_PATH)

In [32]:
swap_df

,reference,instrument,model,date,expiration,tenor,-200,-100,-50,-25,0,25,50,100,200
0,SOFR,swaption,black,2025-01-31,1,4,42.13,31.51,28.15,26.89,25.9,25.16,24.66,24.23,24.73


Your data: ywill use a single row of this data for the `1x4` swaption.
* date: `2025-01-31`
* expiration: 1yr
* tenor: 4yrs

## Rate Data

The file `data/cap_curves_2025-01-31.xlsx` gives 
* SOFR swap rates, 
* their associated discount factors
* their associated forward interest rates.

You will not need the cap data (flat or forward vols) for this problem.

In [33]:
RATES_FILE_PATH = '../data/cap_curves_2025-01-31.xlsx'

rates_df = pd.read_excel(RATES_FILE_PATH)

In [34]:
rates_df

,tenor,swap rates,spot rates,discounts,forwards,flat vols,fwd vols
0,0.25,0.042340,0.042340,0.989526,NaN,NaN,NaN
1,0.50,0.041854,0.041852,0.979398,0.041364,0.101500,0.101500
2,0.75,0.041404,0.041397,0.969584,0.040489,0.116946,0.128478
3,1.00,0.041029,0.041018,0.960012,0.039882,0.132391,0.154562
4,1.25,0.040458,0.040438,0.950950,0.038117,0.159421,0.219138
5,1.50,0.040142,0.040117,0.941881,0.038514,0.180856,0.239613
6,1.75,0.039966,0.039940,0.932816,0.038873,0.197446,0.254106
7,2.00,0.039902,0.039877,0.923708,0.039440,0.209941,0.261605
8,2.25,0.039717,0.039688,0.914976,0.038173,0.219090,0.263323
9,2.50,0.039636,0.039606,0.906171,0.038868,0.225643,0.261958


## The Swaption

Consider the following swaption with the following features:
* underlying is a fixed-for-floating (SOFR) swap
* the underlying swap has **quarterly** payment frequency
* this is a **payer** swaption, which gives the holder the option to **pay** the fixed swap rate and receive SOFR.

### 1.1
Calculate the (relevant) forward swap rate. That is, the one-year forward 4-year swap rate.

### 1.2
Price the swaptions at the quoted implied volatilites and corresponding strikes, all using the just-calculated forward swap rate as the underlying.

### 1.3
To consider how the expiration and tenor matter, calculate the prices of a few other swaptions for comparison. 
* No need to get other implied vol quotes--just use the ATM implied vol you have for the swaption above. (Here we are just interested in how Black's formula changes with changes in tenor and expiration.)
* No need to calculate for all the strikes--just do the ATM strike.

Alternate swaptions
* The 3mo x 4yr swaption
* The 2yr x 4yr swaption
* the 1yr x 2yr swaption

Report these values and compare them to the price of the `1y x 4y` swaption.

In [35]:
def calc_fwdswaprate(discounts, Tfwd, Tswap, freqswap):
    freqdisc = round(1/discounts.index.to_series().diff().mean())
    step = round(freqdisc / freqswap)
    
    periods_fwd = discounts.index.get_loc(Tfwd)
    periods_swap = discounts.index.get_loc(Tswap)
    # get exclusive of left and inclusive of right by shifting both by 1
    periods_fwd += step
    periods_swap += 1
    
    fwdswaprate = freqswap * (discounts.loc[Tfwd] - discounts.loc[Tswap]) / discounts.iloc[periods_fwd:periods_swap:step].sum()
    return fwdswaprate

# 1.1 Solution

In [36]:
DATE = '2025-01-31'
freqcurve = 4

SWAP_TYPE = 'SOFR'
QUOTE_STYLE = 'black'
RELATIVE_STRIKE = 0

expry = 1
tenor = 4

freqswap = 4
isPayer=True
N = 100

swap_df.index = ['implied vol']

In [37]:
rates_df = rates_df.set_index('tenor')

Topt = expry
Tswap = Topt+tenor

fwdrate = rates_df['forwards'][Topt]

fwdswap = calc_fwdswaprate(rates_df['discounts'], Topt, Tswap, freqswap=freqswap)

In [30]:
# 1.1 solution

fwdswap

0.039385244742527234

# 1.2 Solution

In [20]:
def blacks_formula(T,vol,strike,fwd,discount=1,isCall=True):
        
    sigT = vol * np.sqrt(T)
    d1 = (1/sigT) * np.log(fwd/strike) + .5*sigT
    d2 = d1-sigT
    
    if isCall:
        val = discount * (fwd * norm.cdf(d1) - strike * norm.cdf(d2))
    else:
        val = discount * (strike * norm.cdf(-d2) - fwd * norm.cdf(-d1))
    return val

In [38]:
swap_df

,reference,instrument,model,date,expiration,tenor,-200,-100,-50,-25,0,25,50,100,200
implied vol,SOFR,swaption,black,2025-01-31,1,4,42.13,31.51,28.15,26.89,25.9,25.16,24.66,24.23,24.73


In [39]:
strikerange = np.array(swap_df.columns[-9:].tolist())
vols = swap_df[strikerange]
vols /= 100

strikes = fwdswap + strikerange/ 10_000

idstrike = np.where(strikerange==0)[0][0]

idstrikeATM = np.where(strikerange==0)[0][0]

capvol = rates_df.loc[Topt, 'fwd vols']

strikeATM = strikes[idstrikeATM]
volATM = vols.iloc[0, idstrikeATM]

In [40]:
period_fwd = rates_df.index.get_loc(Topt)
period_swap = rates_df.index.get_loc(Tswap)+1
step = round(freqcurve/freqswap)

discount = rates_df['discounts'].iloc[period_fwd+step : period_swap : step].sum()/freqswap
blacks_quotes = vols.copy()
blacks_quotes.loc['price'] = N * blacks_formula(Topt,vols,strikes,fwdswap,discount,isCall=isPayer)[0]
blacks_quotes.loc['strike'] = strikes
blacks_quotes = blacks_quotes.loc[['strike','implied vol','price']]

blacks_quotes.style.format('{:.2%}').format('{:.2f}',subset=pd.IndexSlice['price',:])

,-200,-100,-50,-25,0,25,50,100,200
strike,1.94%,2.94%,3.44%,3.69%,3.94%,4.19%,4.44%,4.94%,5.94%
implied vol,42.13%,31.51%,28.15%,26.89%,25.90%,25.16%,24.66%,24.23%,24.73%
price,7.16,3.90,2.51,1.93,1.44,1.04,0.74,0.36,0.08


# 1.3 Solution

In [41]:
expiries = [.25,1,2,1]
tenors = [4,4,4,2]
fwdswaps = np.full(len(expiries),np.nan)

blacks_quotes_alt = pd.DataFrame(dtype=float,columns=['expiry','tenor','price'])
    
for i in range(len(fwdswaps)):
    fwdswaps[i] = calc_fwdswaprate(rates_df['discounts'], expiries[i], expiries[i]+tenors[i], freqswap=freqswap)
    
    period0 = rates_df.index.get_loc(expiries[i])
    period1 = rates_df.index.get_loc(expiries[i]+tenors[i])+1
    step_i = round(freqcurve/freqswap)

    discount_i = rates_df['discounts'].iloc[period0+step_i : period1 : step_i].sum()/freqswap

    blacks_quotes_alt.loc[i,['expiry','tenor']] = [expiries[i],tenors[i]]
    blacks_quotes_alt.loc[i,'price'] = N * blacks_formula(expiries[i],volATM,strikeATM,fwdswaps[i],discount_i,isCall=isPayer)

    
def highlight_row(row):
    if row.name == 1:
        return ['background-color: gray'] * len(row)
    else:
        return [''] * len(row)
    
blacks_quotes_alt.style.apply(highlight_row,axis=1).format({'expiry':'{:.2f}', 'tenor':'{:.2f}', 'price':'{:.2f}'})

,expiry,tenor,price
0,0.25,4.00,0.76
1,1.00,4.00,1.44
2,2.00,4.00,2.03
3,1.00,2.00,0.70


***

# 2. SABR Volatility Modeling

Use the quoted volatility skew to fit a SABR model.
* Throughout, parameterize, `beta=.75`.

### 2.1.
Estimate $(\alpha,\rho,\nu)$ via the SABR formula. Feel free to use the `sabr` function in `cmds/volskew.py`.

Report the values of these parameters.

### 2.2.
Create a grid of strikes of `[.0025, .09]`, with grid spacing of `10bps`, (.0010).

Use the SABR model parameterized above to calculate the volatility for each of these strikes.

* Plot the SABR curve, and also include the market quotes in the plot.

* Conceptually, does the SABR curve fit these points well? Perfectly?

### 2.3.
Suppose we want to price the `1y4y` swaption but with a far out-of-the-money strike of `5%`. 

* Use the SABR vol at this strike to price the swaption.

(Note that this strike is far outside the range for which we have market quotes, and even if we do have quotes, they likely are not liquid on a given day.)



### 2.4.

Use the ATM implied volatility to fit $\alpha$, (sometimes denoted $\sigma_0$.) That is, for any choice of $(\rho,\nu)$, solve a quadratic equation to get $\alpha$ as a function of those two parameters, along with $\beta$, which is at its selected (not estimated) value.

Recall that we have a simple relationship between ATM vol and $\alpha$.
$$\sigma_\text{ATM} = \alpha\frac{B}{F^{1-\beta}}$$
where $B$ is defined in the discussion note. It is a quadratic expression of $\alpha$ along with $(\beta,\rho,\nu)$.

This decouples the optimization. We solve for $(\rho,\nu)$ and $\alpha$ as a function of the other two parameters.

Consider using the function `sabrATM` in `cmds/volskew.py`.

***

# 3. SABR and Risk Management

Consider how an **increase** of `50bps` in the underlying rate impacts the price of the `1y4y` ATM swaption.

Here, we are assuming that
* the rate change will impact the forward swap rate directly, one-for-one with other rates.
* the shift happens one week after the original quote date.

Of course, in reality, a rate change
* may price in early to some degree if it is expected
* may not impact the forward swap rate one-for-one. We would need to model how the discount curve changes and what that change (level, slope, curvature?) would do to the forward swap rate.

### 3.1

Use Black's equation to re-price the `1y4y` swaption with
* the same ATM volatility
* an underlying (forward swap) rate `increased` `50bps`.
* the same time-to-maturities. (We should decrease these all by 1/365, but we focus here on the delta effects rather than the theta. And it would be a small impact anyone.)

Report 
* the new price
* the change in price divided by `50bps`, (a numerical delta.)

### 3.2

Now, we consider how the volatility may change with the underlying shift `up` of `50bps`.

Using the same SABR parameters from `2.1`, and the strike grid from `2.2`, plot the new SABR curve accounting for the underlying rate shifting `up` `50bps`.

### 3.3

Calculate the new volatility specifically for the pre-shift ATM strike.

Use this in Black's formula similar to `3.1`, where the only change is the volatility now reflects the shift in the underlying (not the strike).

Report
* the new price
* the change in price divided by `50bps`

### 3.4

How much different is the (dynamic) delta which accounts for the shift in volatility from the (static) delta in `3.1`?

***